# ***Importing all required libraries***

In [5]:
import pandas as pd
import numpy as np
import gensim.downloader as api
from bs4 import BeautifulSoup
import re
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt_tab')
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.svm import LinearSVC
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch import nn
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


# ***Dataset Preparation***

## Reading the data

In [6]:
data = pd.read_csv("amazon_reviews_us_Office_Products_v1_00.tsv", sep='\t', on_bad_lines='skip')

<ipython-input-6-0adcb062aa40>:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("amazon_reviews_us_Office_Products_v1_00.tsv", sep='\t', on_bad_lines='skip')


In [7]:
data

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,43081963,R18RVCKGH1SSI9,B001BM2MAC,307809868,"Scotch Cushion Wrap 7961, 12 Inches x 100 Feet",Office Products,5,0.0,0.0,N,Y,Five Stars,Great product.,2015-08-31
1,US,10951564,R3L4L6LW1PUOFY,B00DZYEXPQ,75004341,"Dust-Off Compressed Gas Duster, Pack of 4",Office Products,5,0.0,1.0,N,Y,"Phffffffft, Phfffffft. Lots of air, and it's C...",What's to say about this commodity item except...,2015-08-31
2,US,21143145,R2J8AWXWTDX2TF,B00RTMUHDW,529689027,Amram Tagger Standard Tag Attaching Tagging Gu...,Office Products,5,0.0,0.0,N,Y,but I am sure I will like it.,"Haven't used yet, but I am sure I will like it.",2015-08-31
3,US,52782374,R1PR37BR7G3M6A,B00D7H8XB6,868449945,AmazonBasics 12-Sheet High-Security Micro-Cut ...,Office Products,1,2.0,3.0,N,Y,and the shredder was dirty and the bin was par...,Although this was labeled as &#34;new&#34; the...,2015-08-31
4,US,24045652,R3BDDDZMZBZDPU,B001XCWP34,33521401,"Derwent Colored Pencils, Inktense Ink Pencils,...",Office Products,4,0.0,0.0,N,Y,Four Stars,Gorgeous colors and easy to use,2015-08-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2640249,US,53005790,RLI7EI10S7SN0,B00000DM9M,223408988,PalmOne III Leather Belt Clip Case,Office Products,4,26.0,26.0,N,N,Great value! A must if you hate to carry thing...,I can't live anymore whithout my Palm III. But...,1998-12-07
2640250,US,52188548,R1F3SRK9MHE6A3,B00000DM9M,223408988,PalmOne III Leather Belt Clip Case,Office Products,4,18.0,18.0,N,N,Attaches the Palm Pilot like an appendage,Although the Palm Pilot is thin and compact it...,1998-11-30
2640251,US,52090046,R23V0C4NRJL8EM,0807865001,307284585,Gods and Heroes of Ancient Greece,Office Products,4,9.0,16.0,N,N,"Excellent information, pictures and stories, I...",This book had a lot of great content without b...,1998-10-15
2640252,US,52503173,R13ZAE1ATEUC1T,1572313188,870359649,Microsoft EXCEL 97/ Visual Basic Step-by-Step ...,Office Products,5,0.0,0.0,N,N,class text,I am teaching a course in Excel and am using t...,1998-08-22


## Keeping only two required columns in the input data:

*   review_body
*   star_rating

In [8]:
input_data = data[['review_body', 'star_rating']]

In [9]:
input_data

,review_body,star_rating
0,Great product.,5
1,What's to say about this commodity item except...,5
2,"Haven't used yet, but I am sure I will like it.",5
3,Although this was labeled as &#34;new&#34; the...,1
4,Gorgeous colors and easy to use,4
...,...,...
2640249,I can't live anymore whithout my Palm III. But...,4
2640250,Although the Palm Pilot is thin and compact it...,4
2640251,This book had a lot of great content without b...,4
2640252,I am teaching a course in Excel and am using t...,5


## Dropping the null values.

In [10]:
input_data = input_data.dropna()

## Converting the star ratings in the dataset from their current data type to integer values.

In [11]:
input_data['star_rating'] = input_data['star_rating'].astype(int)

<ipython-input-11-29a186775883>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_data['star_rating'] = input_data['star_rating'].astype(int)


## In the below code:
*   Calculating and displaying the frequency distribution of reviews across different star ratings in the dataset.
*   It groups all reviews by their star rating (1-5) and counts how many reviews exist for each rating level, providing a clear view of the dataset's rating distribution.

In [12]:
print(input_data.groupby('star_rating')['review_body'].count())

star_rating
1     306967
2     138381
3     193680
4     418348
5    1582704
Name: review_body, dtype: int64


## In the below code:

*   Creating a balanced dataset by randomly sampling exactly 50,000 reviews from each star rating category.
*   It first groups the reviews by their star ratings, then randomly selects 50,000 reviews from each group without replacement (meaning no review is selected twice).

In [13]:
balanced_data_1 = input_data.groupby('star_rating').apply(lambda x: x.sample(n=50000, replace=False))

<ipython-input-13-e4e6db7410d4>:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  balanced_data_1 = input_data.groupby('star_rating').apply(lambda x: x.sample(n=50000, replace=False))


In [14]:
balanced_data_1

review_body  \
star_rating                                                              
1           948552   I returned the cards. I found the quality poor...   
            2232175  This was a waste of money. The 1st laster ship...   
            2155672  I purchased this allegedly smokeless ashtray b...   
            2002748  I bought a bunch of these supposedly compatibl...   
            516088   it stopped working not even ten minutes into m...   
...                                                                ...   
5           485248   Exactly as advertised, Thank you for the produ...   
            1730433  I was looking for a projector hanger that had ...   
            1027593  These are exactly what I wanted! Hobby Lobby q...   
            1124563               Its a pen refill.  works as expected   
            1239851                        Still working months later.   

                     star_rating  
star_rating                       
1           948552             1  
            2232175            1  
            2155672            1  
            2002748            1  
            516088             1  
...                          ...  
5           485248             5  
            1730433            5  
            1027593            5  
            1124563            5  
            1239851            5  

[250000 rows x 2 columns]

## Our groupby operation earlier, 'star_rating' became both an index and a column.By resetting the index first, we avoid this ambiguity.

In [15]:
balanced_data_1 = balanced_data_1.reset_index(drop=True)

## In the below code:

*   We assume that ratings more than 3 denote positive sentiment (class 1) and rating less than 3 denote negative sentiment (class
2).
*  Reviews with rating 3 are considered to have neutral sentiment (class 3).



In [16]:

#Positive Sentiment -> 1 and Negative Sentiment -> 2
balanced_data_1['sentiment'] = np.where(balanced_data_1['star_rating'] > 3, 1,
                                      np.where(balanced_data_1['star_rating'] < 3, 2,
                                             3))

In [17]:
balanced_data_1

,review_body,star_rating,sentiment
0,I returned the cards. I found the quality poor...,1,2
1,This was a waste of money. The 1st laster ship...,1,2
2,I purchased this allegedly smokeless ashtray b...,1,2
3,I bought a bunch of these supposedly compatibl...,1,2
4,it stopped working not even ten minutes into m...,1,2
...,...,...,...
249995,"Exactly as advertised, Thank you for the produ...",5,1
249996,I was looking for a projector hanger that had ...,5,1
249997,These are exactly what I wanted! Hobby Lobby q...,5,1
249998,Its a pen refill. works as expected,5,1


## The joint distribution of reviews across both star ratings and sentiment categories.

In [18]:
print(balanced_data_1.groupby(['star_rating', 'sentiment']).size())

star_rating  sentiment
1            2            50000
2            2            50000
3            3            50000
4            1            50000
5            1            50000
dtype: int64


# ***Word Embedding***

## The code loads a pre-trained Word2Vec model, "word2vec-google-news-300", which provides 300-dimensional vector representations of words for use in natural language processing tasks.

In [19]:
wv = api.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [20]:
wv['whale']

array([ 0.08154297,  0.41992188, -0.44921875, -0.01794434, -0.24414062,
       -0.21386719, -0.16796875, -0.01831055,  0.32421875, -0.09228516,
       -0.11523438, -0.5390625 , -0.00637817, -0.41601562, -0.02758789,
        0.04394531, -0.15039062, -0.05712891, -0.03344727, -0.10791016,
        0.14453125,  0.17480469,  0.18847656,  0.02282715, -0.05688477,
       -0.13964844,  0.01379395,  0.296875  ,  0.53515625, -0.2421875 ,
       -0.22167969,  0.23046875, -0.20507812, -0.23242188,  0.0123291 ,
        0.14746094, -0.12597656,  0.25195312,  0.17871094, -0.00106812,
       -0.07080078,  0.10205078, -0.08154297,  0.25390625,  0.04833984,
       -0.11230469,  0.11962891,  0.19335938,  0.44140625,  0.31445312,
       -0.06835938, -0.04760742,  0.37890625, -0.18554688, -0.03063965,
       -0.00386047,  0.01062012, -0.15527344,  0.40234375, -0.13378906,
       -0.00946045, -0.06103516, -0.08251953, -0.44335938,  0.29101562,
       -0.22753906, -0.29296875, -0.13671875, -0.08349609, -0.25

## Converted all reviews into lower case.

In [21]:
lowered_review_body = balanced_data_1['review_body'].str.lower()
balanced_data_1['review_body'] = lowered_review_body
balanced_data_1

,review_body,star_rating,sentiment
0,i returned the cards. i found the quality poor...,1,2
1,this was a waste of money. the 1st laster ship...,1,2
2,i purchased this allegedly smokeless ashtray b...,1,2
3,i bought a bunch of these supposedly compatibl...,1,2
4,it stopped working not even ten minutes into m...,1,2
...,...,...,...
249995,"exactly as advertised, thank you for the produ...",5,1
249996,i was looking for a projector hanger that had ...,5,1
249997,these are exactly what i wanted! hobby lobby q...,5,1
249998,its a pen refill. works as expected,5,1


## Removed the HTML and URLs from the reviews.

In [22]:
balanced_data_1['review_body'] = balanced_data_1['review_body'].apply(lambda x: BeautifulSoup(x, "html.parser").get_text())
balanced_data_1['review_body'] = balanced_data_1['review_body'].str.replace(r'http\S+', '', regex=True)
balanced_data_1

<ipython-input-22-51a70afa5378>:1: MarkupResemblesLocatorWarning: The input passed in on this line looks more like a URL than HTML or XML.

If you meant to use Beautiful Soup to parse the web page found at a certain URL, then something has gone wrong. You should use an Python package like 'requests' to fetch the content behind the URL. Once you have the content as a string, you can feed that string into Beautiful Soup.

However, if you want to parse some data that happens to look like a URL, then nothing has gone wrong: you are using Beautiful Soup correctly, and this warning is spurious and can be filtered. To make this warning go away, run this code before calling the BeautifulSoup constructor:

    from bs4 import MarkupResemblesLocatorWarning
    import warnings

    warnings.filterwarnings("ignore", category=MarkupResemblesLocatorWarning)
    
  balanced_data_1['review_body'] = balanced_data_1['review_body'].apply(lambda x: BeautifulSoup(x, "html.parser").get_text())


,review_body,star_rating,sentiment
0,i returned the cards. i found the quality poor...,1,2
1,this was a waste of money. the 1st laster ship...,1,2
2,i purchased this allegedly smokeless ashtray b...,1,2
3,i bought a bunch of these supposedly compatibl...,1,2
4,it stopped working not even ten minutes into m...,1,2
...,...,...,...
249995,"exactly as advertised, thank you for the produ...",5,1
249996,i was looking for a projector hanger that had ...,5,1
249997,these are exactly what i wanted! hobby lobby q...,5,1
249998,its a pen refill. works as expected,5,1


## Performed contractions on the reviews, e.g., won’t → will not. Included as many contractions in English that I can think of.

In [23]:
contractions_dict = {
    "don't": "do not",
    "can't": "cannot",
    "won't": "will not",
    "i'm": "i am",
    "it's": "it is",
    "isn't": "is not",
    "aren't": "are not",
    "wasn't": "was not",
    "weren't": "were not",
    "haven't": "have not",
    "hasn't": "has not",
    "hadn't": "had not",
    "let's": "let us",
    "you'd": "you would",
    "you'll": "you will",
    "you've": "you have",
    "we're": "we are",
    "we'll": "we will",
    "they're": "they are",
    "they've": "they have",
    "he's": "he is",
    "she's": "she is",
    "that's": "that is",
    "there's": "there is",
    "what's": "what is",
    "here's": "here is",
    "who's": "who is",
    "how's": "how is",
    "i've": "i have",
    "i'd": "i would",
    "we'd": "we would",
    "they'd": "they would",
    "it'd": "it would",
    "he'd": "he would",
    "she'd": "she would",
    "doesn't": "does not",
    "didn't": "did not",
    "couldn't": "could not",
    "shouldn't": "should not",
    "mightn't": "might not",
    "mustn't": "must not",
}

In [24]:

contractions = re.compile('|'.join(re.escape(key) for key in contractions_dict.keys()))
balanced_data_1['review_body'] = balanced_data_1['review_body'].apply(
    lambda x: contractions.sub(lambda match: contractions_dict[match.group(0)], x)
)


In [25]:
balanced_data_1

,review_body,star_rating,sentiment
0,i returned the cards. i found the quality poor...,1,2
1,this was a waste of money. the 1st laster ship...,1,2
2,i purchased this allegedly smokeless ashtray b...,1,2
3,i bought a bunch of these supposedly compatibl...,1,2
4,it stopped working not even ten minutes into m...,1,2
...,...,...,...
249995,"exactly as advertised, thank you for the produ...",5,1
249996,i was looking for a projector hanger that had ...,5,1
249997,these are exactly what i wanted! hobby lobby q...,5,1
249998,its a pen refill. works as expected,5,1


## Removed non-alphabetical characters.stop words are often removed from text analysis as they don't typically carry much meaning (such as "the," "a," "is").

In [26]:
balanced_data_1['review_body'] = balanced_data_1['review_body'].str.replace(r'[^a-z\s]','', regex=True)
balanced_data_1

,review_body,star_rating,sentiment
0,i returned the cards i found the quality poor ...,1,2
1,this was a waste of money the st laster shippe...,1,2
2,i purchased this allegedly smokeless ashtray b...,1,2
3,i bought a bunch of these supposedly compatibl...,1,2
4,it stopped working not even ten minutes into m...,1,2
...,...,...,...
249995,exactly as advertised thank you for the produc...,5,1
249996,i was looking for a projector hanger that had ...,5,1
249997,these are exactly what i wanted hobby lobby qu...,5,1
249998,its a pen refill works as expected,5,1


## Removed extra spaces

In [27]:
balanced_data_1['review_body'] = balanced_data_1['review_body'].str.replace(' +', ' ', regex=True)
balanced_data_1

,review_body,star_rating,sentiment
0,i returned the cards i found the quality poor ...,1,2
1,this was a waste of money the st laster shippe...,1,2
2,i purchased this allegedly smokeless ashtray b...,1,2
3,i bought a bunch of these supposedly compatibl...,1,2
4,it stopped working not even ten minutes into m...,1,2
...,...,...,...
249995,exactly as advertised thank you for the produc...,5,1
249996,i was looking for a projector hanger that had ...,5,1
249997,these are exactly what i wanted hobby lobby qu...,5,1
249998,its a pen refill works as expected,5,1


## This code imports text processing tools from the NLTK library:

*   WordNetLemmatizer for reducing words to their base form.
*   stopwords for filtering out common words like "the" and "is,".
*  word_tokenize for splitting text into individual words.


It then downloads necessary datasets (stopwords and punkt_tab) and initializes a lemmatizer while defining a set of English stopwords for text preprocessing.

In [28]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [29]:
balanced_data_1

,review_body,star_rating,sentiment
0,i returned the cards i found the quality poor ...,1,2
1,this was a waste of money the st laster shippe...,1,2
2,i purchased this allegedly smokeless ashtray b...,1,2
3,i bought a bunch of these supposedly compatibl...,1,2
4,it stopped working not even ten minutes into m...,1,2
...,...,...,...
249995,exactly as advertised thank you for the produc...,5,1
249996,i was looking for a projector hanger that had ...,5,1
249997,these are exactly what i wanted hobby lobby qu...,5,1
249998,its a pen refill works as expected,5,1


In [30]:
def preprocess_text(text):
    if not isinstance(text, str):
        text = ' '.join(text) if isinstance(text, list) else str(text)
    tokens = text.split()
    tokens = [word for word in tokens if word.lower() not in stop_words]
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(tokens)

In [31]:
balanced_data_1['review_body'] = balanced_data_1['review_body'].astype(str)
balanced_data_1['review_body'] = balanced_data_1['review_body'].apply(preprocess_text)

In [32]:
balanced_data_1

,review_body,star_rating,sentiment
0,returned card found quality poor flimsy inside...,1,2
1,waste money st laster shipped didnt work took ...,1,2
2,purchased allegedly smokeless ashtray house gu...,1,2
3,bought bunch supposedly compatible cartridge a...,1,2
4,stopped working even ten minute using,1,2
...,...,...,...
249995,exactly advertised thank product timely delivery,5,1
249996,looking projector hanger smallest drop since u...,5,1
249997,exactly wanted hobby lobby quit carrying glad ...,5,1
249998,pen refill work expected,5,1


In [33]:
balanced_data_1['tokens'] = balanced_data_1['review_body'].apply(word_tokenize)

In [34]:
balanced_data_1

,review_body,star_rating,sentiment,tokens
0,returned card found quality poor flimsy inside...,1,2,"[returned, card, found, quality, poor, flimsy,..."
1,waste money st laster shipped didnt work took ...,1,2,"[waste, money, st, laster, shipped, didnt, wor..."
2,purchased allegedly smokeless ashtray house gu...,1,2,"[purchased, allegedly, smokeless, ashtray, hou..."
3,bought bunch supposedly compatible cartridge a...,1,2,"[bought, bunch, supposedly, compatible, cartri..."
4,stopped working even ten minute using,1,2,"[stopped, working, even, ten, minute, using]"
...,...,...,...,...
249995,exactly advertised thank product timely delivery,5,1,"[exactly, advertised, thank, product, timely, ..."
249996,looking projector hanger smallest drop since u...,5,1,"[looking, projector, hanger, smallest, drop, s..."
249997,exactly wanted hobby lobby quit carrying glad ...,5,1,"[exactly, wanted, hobby, lobby, quit, carrying..."
249998,pen refill work expected,5,1,"[pen, refill, work, expected]"


## The below function get_sentence_embedding generates sentence embeddings by averaging the word embeddings of the individual words in the sentence. If a word is not found in the pre-trained Word2Vec model, it's ignored. This function is applied to the 'tokens' column of the DataFrame balanced_data_1 to create a new column 'Pre-trained embeddings' storing these sentence embeddings.

In [35]:
def get_sentence_embedding(tokens):
    word_vectors = [wv[word] for word in tokens if word in wv]

    if word_vectors:
        return np.mean(word_vectors, axis=0)
    else:
        return np.zeros(300)

balanced_data_1['Pre-trained embeddings'] = balanced_data_1['tokens'].apply(get_sentence_embedding)

In [36]:
balanced_data_1

,review_body,star_rating,sentiment,tokens,Pre-trained embeddings
0,returned card found quality poor flimsy inside...,1,2,"[returned, card, found, quality, poor, flimsy,...","[0.06468335, 0.06768478, -0.02350148, 0.075450..."
1,waste money st laster shipped didnt work took ...,1,2,"[waste, money, st, laster, shipped, didnt, wor...","[0.041154534, 0.06830432, 0.029785682, 0.06836..."
2,purchased allegedly smokeless ashtray house gu...,1,2,"[purchased, allegedly, smokeless, ashtray, hou...","[0.06975949, 0.04399525, 0.015591708, 0.076345..."
3,bought bunch supposedly compatible cartridge a...,1,2,"[bought, bunch, supposedly, compatible, cartri...","[-0.0014282226, 0.048101425, 0.03824692, 0.088..."
4,stopped working even ten minute using,1,2,"[stopped, working, even, ten, minute, using]","[0.020268759, 0.03565216, 0.04871623, 0.083923..."
...,...,...,...,...,...
249995,exactly advertised thank product timely delivery,5,1,"[exactly, advertised, thank, product, timely, ...","[-0.06542969, -0.02557373, -0.014322917, 0.007..."
249996,looking projector hanger smallest drop since u...,5,1,"[looking, projector, hanger, smallest, drop, s...","[0.020986496, -0.013807727, -0.029896397, 0.08..."
249997,exactly wanted hobby lobby quit carrying glad ...,5,1,"[exactly, wanted, hobby, lobby, quit, carrying...","[0.02428521, 0.05687098, -0.0034518773, 0.0550..."
249998,pen refill work expected,5,1,"[pen, refill, work, expected]","[-0.059318542, 0.15197754, -0.10107422, 0.0948..."


In [37]:
print(balanced_data_1[['review_body', 'Pre-trained embeddings']].head())

                                         review_body  \
0  returned card found quality poor flimsy inside...   
1  waste money st laster shipped didnt work took ...   
2  purchased allegedly smokeless ashtray house gu...   
3  bought bunch supposedly compatible cartridge a...   
4              stopped working even ten minute using   

                              Pre-trained embeddings  
0  [0.06468335, 0.06768478, -0.02350148, 0.075450...  
1  [0.041154534, 0.06830432, 0.029785682, 0.06836...  
2  [0.06975949, 0.04399525, 0.015591708, 0.076345...  
3  [-0.0014282226, 0.048101425, 0.03824692, 0.088...  
4  [0.020268759, 0.03565216, 0.04871623, 0.083923...  


## The below function checks how similar two pieces of text are by comparing the individual words within them using a pre-trained model. If there's enough overlap in the words used, it calculates an average similarity score, otherwise, it assumes no similarity.

In [38]:
def get_review_similarity(review1_tokens, review2_tokens, word2vec_model):
    review1_tokens = [token for token in review1_tokens if token in word2vec_model]
    review2_tokens = [token for token in review2_tokens if token in word2vec_model]

    if not review1_tokens or not review2_tokens:
        return 0.0

    similarities = []
    for word1 in review1_tokens:
        for word2 in review2_tokens:
            try:
                sim = word2vec_model.similarity(word1, word2)
                similarities.append(sim)
            except KeyError:
                continue
    return np.mean(similarities) if similarities else 0.0

In [39]:
review1_idx = 0
review2_idx = 1

In [40]:
tokens1 = balanced_data_1['tokens'].iloc[review1_idx]
tokens2 = balanced_data_1['tokens'].iloc[review2_idx]

## This code compares two specific reviews and prints them alongside a score showing how similar their content is, based on the words they use.

In [41]:
similarity = get_review_similarity(tokens1, tokens2, wv)

print("Review 1:", balanced_data_1['review_body'].iloc[review1_idx])
print("Review 2:", balanced_data_1['review_body'].iloc[review2_idx])
print("\nSimilarity Score:", similarity)

Review 1: returned card found quality poor flimsy inside yellowishcream color gave feeling old stock front card white lettering andor embellishment inside yellowish color work inexpensive lesson learned get pay sometimes hanukkah card difficult find store live fact able locate amazon card appeared liking yes cost little one last note holiday many quality card sale price right youre ready next year
Review 2: waste money st laster shipped didnt work took another week company ship another one many email sent nd one came stopped working day im keeping dont feel like dealing hassle

Similarity Score: 0.109699726


## The below function finds the most similar pairs of words between two pieces of text, using the pre-trained model to measure the similarity. It filters out words not found in the model and returns a list of the top topn most similar word pairs.

In [42]:
def find_similar_words_between_reviews(tokens1, tokens2, word2vec_model, topn=5):
    tokens1 = [token for token in tokens1 if token in word2vec_model]
    tokens2 = [token for token in tokens2 if token in word2vec_model]

    word_pairs = []
    for word1 in tokens1:
        for word2 in tokens2:
            try:
                sim = word2vec_model.similarity(word1, word2)
                word_pairs.append((word1, word2, sim))
            except KeyError:
                continue

    return sorted(word_pairs, key=lambda x: x[2], reverse=True)[:topn]

In [43]:
similar_pairs = find_similar_words_between_reviews(tokens1, tokens2, wv)

## This code displays the most similar word pairs and their similarity scores that were previously found between two reviews.

In [44]:
print("\nMost similar word pairs between the reviews:")
for word1, word2, sim in similar_pairs:
    print(f"'{word1}' - '{word2}': {sim:.4f}")


Most similar word pairs between the reviews:
'work' - 'work': 1.0000
'many' - 'many': 1.0000
'one' - 'one': 1.0000
'one' - 'one': 1.0000
'feeling' - 'feel': 0.7652


## Training the Custom Model

In [45]:
balanced_data_1

,review_body,star_rating,sentiment,tokens,Pre-trained embeddings
0,returned card found quality poor flimsy inside...,1,2,"[returned, card, found, quality, poor, flimsy,...","[0.06468335, 0.06768478, -0.02350148, 0.075450..."
1,waste money st laster shipped didnt work took ...,1,2,"[waste, money, st, laster, shipped, didnt, wor...","[0.041154534, 0.06830432, 0.029785682, 0.06836..."
2,purchased allegedly smokeless ashtray house gu...,1,2,"[purchased, allegedly, smokeless, ashtray, hou...","[0.06975949, 0.04399525, 0.015591708, 0.076345..."
3,bought bunch supposedly compatible cartridge a...,1,2,"[bought, bunch, supposedly, compatible, cartri...","[-0.0014282226, 0.048101425, 0.03824692, 0.088..."
4,stopped working even ten minute using,1,2,"[stopped, working, even, ten, minute, using]","[0.020268759, 0.03565216, 0.04871623, 0.083923..."
...,...,...,...,...,...
249995,exactly advertised thank product timely delivery,5,1,"[exactly, advertised, thank, product, timely, ...","[-0.06542969, -0.02557373, -0.014322917, 0.007..."
249996,looking projector hanger smallest drop since u...,5,1,"[looking, projector, hanger, smallest, drop, s...","[0.020986496, -0.013807727, -0.029896397, 0.08..."
249997,exactly wanted hobby lobby quit carrying glad ...,5,1,"[exactly, wanted, hobby, lobby, quit, carrying...","[0.02428521, 0.05687098, -0.0034518773, 0.0550..."
249998,pen refill work expected,5,1,"[pen, refill, work, expected]","[-0.059318542, 0.15197754, -0.10107422, 0.0948..."


## This code trains a new word embedding model on the provided review data, where each word is represented by a 300-dimensional vector. The model learns these vectors by considering the context of each word within a window of 11 words. It ignores words appearing less than 10 times and utilizes 4 CPU cores for faster training. It uses the skip-gram algorithm, which predicts surrounding words based on the current word.

In [ ]:
corpus_split = balanced_data_1['tokens'].tolist()
model = Word2Vec(
    sentences=corpus_split,
    vector_size=300,
    window=11,
    min_count=10,
    workers=4,
    sg=1
)

## Saving and loading the custom model

In [ ]:
model.save("my_word2vec.model")

In [48]:
model = Word2Vec.load("my_word2vec.model")

## The below function generates sentence embeddings using your custom-trained Word2Vec model. It does this by averaging the vector representations of all the words in the sentence. If a word doesn't exist in your model's vocabulary, it's skipped. If no words in the sentence are found in the vocabulary, it returns a vector of all zeros.

In [49]:
def get_custom_embedding(tokens, model, vector_size=300):
    word_vectors = [model.wv[word] for word in tokens if word in model.wv]

    if word_vectors:
        return np.mean(word_vectors, axis=0)
    else:
        return np.zeros(vector_size)

In [50]:
balanced_data_1['custom_embeddings'] = balanced_data_1['tokens'].apply(lambda x: get_custom_embedding(x, model))

In [51]:
print(balanced_data_1[['tokens', 'custom_embeddings']].head())

                                              tokens  \
0  [returned, card, found, quality, poor, flimsy,...   
1  [waste, money, st, laster, shipped, didnt, wor...   
2  [purchased, allegedly, smokeless, ashtray, hou...   
3  [bought, bunch, supposedly, compatible, cartri...   
4       [stopped, working, even, ten, minute, using]   

                                   custom_embeddings  
0  [-0.00865445, 0.08607587, 0.010163504, 0.08254...  
1  [-0.046187904, 0.12178713, 0.0056259334, 0.107...  
2  [0.008813707, 0.17818084, -0.0925688, 0.121685...  
3  [0.025674889, 0.034872424, -0.117311954, 0.030...  
4  [0.022188505, 0.13320307, -0.059528697, -0.051...  


In [52]:
balanced_data_1

,review_body,star_rating,sentiment,tokens,Pre-trained embeddings,custom_embeddings
0,returned card found quality poor flimsy inside...,1,2,"[returned, card, found, quality, poor, flimsy,...","[0.06468335, 0.06768478, -0.02350148, 0.075450...","[-0.00865445, 0.08607587, 0.010163504, 0.08254..."
1,waste money st laster shipped didnt work took ...,1,2,"[waste, money, st, laster, shipped, didnt, wor...","[0.041154534, 0.06830432, 0.029785682, 0.06836...","[-0.046187904, 0.12178713, 0.0056259334, 0.107..."
2,purchased allegedly smokeless ashtray house gu...,1,2,"[purchased, allegedly, smokeless, ashtray, hou...","[0.06975949, 0.04399525, 0.015591708, 0.076345...","[0.008813707, 0.17818084, -0.0925688, 0.121685..."
3,bought bunch supposedly compatible cartridge a...,1,2,"[bought, bunch, supposedly, compatible, cartri...","[-0.0014282226, 0.048101425, 0.03824692, 0.088...","[0.025674889, 0.034872424, -0.117311954, 0.030..."
4,stopped working even ten minute using,1,2,"[stopped, working, even, ten, minute, using]","[0.020268759, 0.03565216, 0.04871623, 0.083923...","[0.022188505, 0.13320307, -0.059528697, -0.051..."
...,...,...,...,...,...,...
249995,exactly advertised thank product timely delivery,5,1,"[exactly, advertised, thank, product, timely, ...","[-0.06542969, -0.02557373, -0.014322917, 0.007...","[-0.020638617, 0.12789576, -0.07443294, 0.0957..."
249996,looking projector hanger smallest drop since u...,5,1,"[looking, projector, hanger, smallest, drop, s...","[0.020986496, -0.013807727, -0.029896397, 0.08...","[0.0045630815, 0.110500075, 0.020787884, 0.121..."
249997,exactly wanted hobby lobby quit carrying glad ...,5,1,"[exactly, wanted, hobby, lobby, quit, carrying...","[0.02428521, 0.05687098, -0.0034518773, 0.0550...","[-0.01600385, 0.16626655, 0.031086037, 0.07007..."
249998,pen refill work expected,5,1,"[pen, refill, work, expected]","[-0.059318542, 0.15197754, -0.10107422, 0.0948...","[-0.13120882, 0.0047933124, -0.079583794, 0.03..."


## The below function checks how similar two pieces of text are by comparing the individual words within them using a custom trained Word2Vec model. If there's enough overlap in the words used, it calculates an average similarity score, otherwise, it assumes no similarity.

In [53]:
def get_review_similarity_custom(review1_tokens, review2_tokens, word2vec_model):
    review1_tokens = [token for token in review1_tokens if token in word2vec_model.wv.key_to_index]
    review2_tokens = [token for token in review2_tokens if token in word2vec_model.wv.key_to_index]

    if not review1_tokens or not review2_tokens:
        return 0.0

    similarities = []
    for word1 in review1_tokens:
        for word2 in review2_tokens:
            try:
                sim = word2vec_model.wv.similarity(word1, word2)
                similarities.append(sim)
            except KeyError:
                continue
    return np.mean(similarities) if similarities else 0.0

In [54]:
review1_idx_for_custom = 0
review2_idx_for_custom = 1

In [55]:
tokens1_for_custom = balanced_data_1['tokens'].iloc[review1_idx]
tokens2_for_custom = balanced_data_1['tokens'].iloc[review2_idx]

## This code compares two specific reviews and prints them alongside a score showing how similar their content is, based on the words they use, using the custom-trained model.

In [56]:
similarity = get_review_similarity_custom(tokens1_for_custom, tokens2_for_custom, model)

print("Review 1:", balanced_data_1['review_body'].iloc[review1_idx_for_custom])
print("Review 2:", balanced_data_1['review_body'].iloc[review2_idx_for_custom])
print("\nSimilarity Score:", similarity)

Review 1: returned card found quality poor flimsy inside yellowishcream color gave feeling old stock front card white lettering andor embellishment inside yellowish color work inexpensive lesson learned get pay sometimes hanukkah card difficult find store live fact able locate amazon card appeared liking yes cost little one last note holiday many quality card sale price right youre ready next year
Review 2: waste money st laster shipped didnt work took another week company ship another one many email sent nd one came stopped working day im keeping dont feel like dealing hassle

Similarity Score: 0.19658946


## The function finds the most similar pairs of words between two pieces of text, using the custom-trained model to measure the similarity. It filters out words not found in the model and returns a list of the top topn most similar word pairs.

In [57]:
def find_similar_words_between_reviews_custom(tokens1, tokens2, word2vec_model, topn=5):
    tokens1 = [token for token in tokens1 if token in word2vec_model.wv.key_to_index]
    tokens2 = [token for token in tokens2 if token in word2vec_model.wv.key_to_index]

    word_pairs = []
    for word1 in tokens1:
        for word2 in tokens2:
            try:
                sim = word2vec_model.wv.similarity(word1, word2)
                word_pairs.append((word1, word2, sim))
            except KeyError:
                continue

    return sorted(word_pairs, key=lambda x: x[2], reverse=True)[:topn]

In [58]:
similar_pairs = find_similar_words_between_reviews_custom(tokens1_for_custom, tokens2_for_custom, model)

## This code displays the most similar word pairs and their similarity scores that were previously found between two reviews, using the custom-trained Word2Vec model.

In [59]:
print("\nMost similar word pairs between the reviews:")
for word1, word2, sim in similar_pairs:
    print(f"'{word1}' - '{word2}': {sim:.4f}")


Most similar word pairs between the reviews:
'work' - 'work': 1.0000
'one' - 'one': 1.0000
'one' - 'one': 1.0000
'many' - 'many': 1.0000
'work' - 'working': 0.6344


In [60]:
result1 = wv.most_similar(
    positive=['bad', 'amazing'],
    negative=['good'],
    topn=1
)
print("Semantic Similarities using pre-trained model:")
print("Amazing - Good + Bad =", result1[0][0])
similarity = wv.similarity('fantastic', 'excellent')
print("fantastic ~ excellent =", round(similarity, 4))

Semantic Similarities using pre-trained model:
Amazing - Good + Bad = incredible
fantastic ~ excellent = 0.6803


In [61]:
result1 = model.wv.most_similar(
    positive=['bad', 'amazing'],
    negative=['good'],
    topn=1
)
print("Semantic Similarities using custom model:")
print("Amazing - Good + Bad =", result1[0][0])
similarity = model.wv.similarity('fantastic', 'excellent')
print("fantastic ~ excellent =", round(similarity, 4))

Semantic Similarities using custom model:
Amazing - Good + Bad = awful
fantastic ~ excellent = 0.6088


## **Comparing the two Word2Vec models:**

Pre-trained Model appears to encode semantic similarities better because:

*   It shows more nuanced similarity scores between reviews (0.109 vs 0.196)
*   Captures more meaningful semantic relationships between words (e.g., 'feeling'-'feel' similarity: 0.7652)
* Shows stronger semantic connection between sentiment-related words (fantastic~excellent: 0.6803)

Custom Model shows limitations:


* Mainly identifies exact word matches (work-work, one-one)
Lower similarity scores for related words (work-working: 0.6344)
* Limited vocabulary and semantic understanding due to being trained on smaller dataset

The pre-trained Google News model demonstrates superior performance in encoding semantic similarities, likely due to its extensive training data and broader vocabulary coverage.

# ***Simple Models***

In [62]:
balanced_data_1

,review_body,star_rating,sentiment,tokens,Pre-trained embeddings,custom_embeddings
0,returned card found quality poor flimsy inside...,1,2,"[returned, card, found, quality, poor, flimsy,...","[0.06468335, 0.06768478, -0.02350148, 0.075450...","[-0.00865445, 0.08607587, 0.010163504, 0.08254..."
1,waste money st laster shipped didnt work took ...,1,2,"[waste, money, st, laster, shipped, didnt, wor...","[0.041154534, 0.06830432, 0.029785682, 0.06836...","[-0.046187904, 0.12178713, 0.0056259334, 0.107..."
2,purchased allegedly smokeless ashtray house gu...,1,2,"[purchased, allegedly, smokeless, ashtray, hou...","[0.06975949, 0.04399525, 0.015591708, 0.076345...","[0.008813707, 0.17818084, -0.0925688, 0.121685..."
3,bought bunch supposedly compatible cartridge a...,1,2,"[bought, bunch, supposedly, compatible, cartri...","[-0.0014282226, 0.048101425, 0.03824692, 0.088...","[0.025674889, 0.034872424, -0.117311954, 0.030..."
4,stopped working even ten minute using,1,2,"[stopped, working, even, ten, minute, using]","[0.020268759, 0.03565216, 0.04871623, 0.083923...","[0.022188505, 0.13320307, -0.059528697, -0.051..."
...,...,...,...,...,...,...
249995,exactly advertised thank product timely delivery,5,1,"[exactly, advertised, thank, product, timely, ...","[-0.06542969, -0.02557373, -0.014322917, 0.007...","[-0.020638617, 0.12789576, -0.07443294, 0.0957..."
249996,looking projector hanger smallest drop since u...,5,1,"[looking, projector, hanger, smallest, drop, s...","[0.020986496, -0.013807727, -0.029896397, 0.08...","[0.0045630815, 0.110500075, 0.020787884, 0.121..."
249997,exactly wanted hobby lobby quit carrying glad ...,5,1,"[exactly, wanted, hobby, lobby, quit, carrying...","[0.02428521, 0.05687098, -0.0034518773, 0.0550...","[-0.01600385, 0.16626655, 0.031086037, 0.07007..."
249998,pen refill work expected,5,1,"[pen, refill, work, expected]","[-0.059318542, 0.15197754, -0.10107422, 0.0948...","[-0.13120882, 0.0047933124, -0.079583794, 0.03..."


## Train Test - Custom Model

In [63]:
X_train_custom, X_test_custom, y_train_custom, y_test_custom = train_test_split(np.stack(balanced_data_1['custom_embeddings'].values), balanced_data_1['sentiment'].isin([1, 2]), test_size=0.2,random_state=40)

In [64]:
print(f"Training set shape: {X_train_custom.shape}, {y_train_custom.shape}")
print(f"Testing set shape: {X_test_custom.shape}, {y_test_custom.shape}")

Training set shape: (200000, 300), (200000,)
Testing set shape: (50000, 300), (50000,)


## Train Test - Pre-trained Model

In [65]:
X_train, X_test, y_train, y_test = train_test_split(np.stack(balanced_data_1['Pre-trained embeddings'].values), balanced_data_1['sentiment'].isin([1, 2]), test_size=0.2,random_state=40)

In [66]:
print(f"Training set shape: {X_train.shape}, {y_train.shape}")
print(f"Testing set shape: {X_test.shape}, {y_test.shape}")

Training set shape: (200000, 300), (200000,)
Testing set shape: (50000, 300), (50000,)


## TF-IDF

In [67]:
tf_idf = TfidfVectorizer()

In [68]:
result = tf_idf.fit_transform(balanced_data_1['review_body'])

In [69]:
X_train_TFIDF, X_test_TFIDF, y_train_TFIDF, y_test_TFIDF = train_test_split(result, balanced_data_1['sentiment'].isin([1, 2]), test_size=0.2,random_state=40)

## Perceptron for custom model

In [70]:
perceptron = Perceptron(max_iter=1200, eta0=0.1, random_state=40)
perceptron.fit(X_train_custom, y_train_custom)

Perceptron(eta0=0.1, max_iter=1200, random_state=40)

In [71]:
y_train_pred = perceptron.predict(X_train_custom)
y_test_pred = perceptron.predict(X_test_custom)

In [72]:
test_accuracy_Perceptron_Custom_Model = accuracy_score(y_test_custom, y_test_pred)

In [73]:
print("Testing Data:")
print(f"Accuracy: {test_accuracy_Perceptron_Custom_Model:.4f}")

Testing Data:
Accuracy: 0.7868


## Perceptron for pre-trained model

In [74]:
perceptron = Perceptron(max_iter=1200, eta0=0.1, random_state=40)
perceptron.fit(X_train, y_train)

Perceptron(eta0=0.1, max_iter=1200, random_state=40)

In [75]:
y_train_pred = perceptron.predict(X_train)
y_test_pred = perceptron.predict(X_test)

In [76]:
test_accuracy_Perceptron_Pretrained_Model = accuracy_score(y_test, y_test_pred)

In [77]:
print("Testing Data:")
print(f"Accuracy: {test_accuracy_Perceptron_Pretrained_Model:.4f}")

Testing Data:
Accuracy: 0.7643


## Perceptron for TF-IDF

In [78]:
perceptron = Perceptron(max_iter=1200, eta0=0.1, random_state=40)
perceptron.fit(X_train_TFIDF, y_train_TFIDF)

Perceptron(eta0=0.1, max_iter=1200, random_state=40)

In [79]:
y_train_pred = perceptron.predict(X_train_TFIDF)
y_test_pred = perceptron.predict(X_test_TFIDF)

In [80]:
test_accuracy_Perceptron_TFIDF = accuracy_score(y_test_TFIDF, y_test_pred)

In [81]:
print("Testing Data:")
print(f"Accuracy: {test_accuracy_Perceptron_TFIDF:.4f}")

Testing Data:
Accuracy: 0.7151


## SVM for Pretrained model

In [82]:
linear_svc = LinearSVC(random_state=40, max_iter=1200)
linear_svc.fit(X_train, y_train)

LinearSVC(max_iter=1200, random_state=40)

In [83]:
y_train_pred = linear_svc.predict(X_train)
y_test_pred = linear_svc.predict(X_test)

In [84]:
test_accuracy_SVM_Pretrained_Model = accuracy_score(y_test, y_test_pred)

In [85]:
print("Testing Metrics:")
print(f"Accuracy: {test_accuracy_SVM_Pretrained_Model:.4f}")

Testing Metrics:
Accuracy: 0.7975


## SVM for Custom model

In [86]:
linear_svc = LinearSVC(random_state=40, max_iter=1200)
linear_svc.fit(X_train_custom, y_train_custom)

LinearSVC(max_iter=1200, random_state=40)

In [87]:
y_train_pred = linear_svc.predict(X_train_custom)
y_test_pred = linear_svc.predict(X_test_custom)

In [88]:
test_accuracy_SVM_Custom_Model = accuracy_score(y_test_custom, y_test_pred)

In [89]:
print("Testing Metrics:")
print(f"Accuracy: {test_accuracy_SVM_Custom_Model:.4f}")

Testing Metrics:
Accuracy: 0.7971


## SVM for TFIDF

In [90]:
linear_svc = LinearSVC(random_state=40, max_iter=1200)
linear_svc.fit(X_train_TFIDF, y_train_TFIDF)

LinearSVC(max_iter=1200, random_state=40)

In [91]:
y_train_pred = linear_svc.predict(X_train_TFIDF)
y_test_pred = linear_svc.predict(X_test_TFIDF)

In [92]:
test_accuracy_SVM_TFIDF = accuracy_score(y_test_TFIDF, y_test_pred)

In [93]:
print("Testing Metrics:")
print(f"Accuracy: {test_accuracy_SVM_TFIDF:.4f}")

Testing Metrics:
Accuracy: 0.7961


## **Comparing the model performances across different feature types:**
**Perceptron:**

* Custom Word2Vec: 0.7868
* Pre-trained Word2Vec: 0.7643
* TF-IDF: 0.7151

**SVM:**

* Pre-trained Word2Vec: 0.7975
* Custom Word2Vec: 0.7971
* TF-IDF: 0.7961

Conclusions:

* SVM consistently outperforms Perceptron across all feature types.
* Performance ranking from best to worst:

  * Word2Vec (both custom and pre-trained) slightly outperforms TF-IDF
  * Custom Word2Vec performs marginally better than pre-trained for Perceptron
  * All feature types perform very similarly for SVM (difference < 0.002)

* The choice of model (SVM vs Perceptron) has more impact on performance than the choice of feature type

* These results suggest that while Word2Vec features offer a small advantage, the classifier choice is more crucial for performance optimization.

# ***Feedforward Neural Networks***

In [94]:
balanced_data_1

,review_body,star_rating,sentiment,tokens,Pre-trained embeddings,custom_embeddings
0,returned card found quality poor flimsy inside...,1,2,"[returned, card, found, quality, poor, flimsy,...","[0.06468335, 0.06768478, -0.02350148, 0.075450...","[-0.00865445, 0.08607587, 0.010163504, 0.08254..."
1,waste money st laster shipped didnt work took ...,1,2,"[waste, money, st, laster, shipped, didnt, wor...","[0.041154534, 0.06830432, 0.029785682, 0.06836...","[-0.046187904, 0.12178713, 0.0056259334, 0.107..."
2,purchased allegedly smokeless ashtray house gu...,1,2,"[purchased, allegedly, smokeless, ashtray, hou...","[0.06975949, 0.04399525, 0.015591708, 0.076345...","[0.008813707, 0.17818084, -0.0925688, 0.121685..."
3,bought bunch supposedly compatible cartridge a...,1,2,"[bought, bunch, supposedly, compatible, cartri...","[-0.0014282226, 0.048101425, 0.03824692, 0.088...","[0.025674889, 0.034872424, -0.117311954, 0.030..."
4,stopped working even ten minute using,1,2,"[stopped, working, even, ten, minute, using]","[0.020268759, 0.03565216, 0.04871623, 0.083923...","[0.022188505, 0.13320307, -0.059528697, -0.051..."
...,...,...,...,...,...,...
249995,exactly advertised thank product timely delivery,5,1,"[exactly, advertised, thank, product, timely, ...","[-0.06542969, -0.02557373, -0.014322917, 0.007...","[-0.020638617, 0.12789576, -0.07443294, 0.0957..."
249996,looking projector hanger smallest drop since u...,5,1,"[looking, projector, hanger, smallest, drop, s...","[0.020986496, -0.013807727, -0.029896397, 0.08...","[0.0045630815, 0.110500075, 0.020787884, 0.121..."
249997,exactly wanted hobby lobby quit carrying glad ...,5,1,"[exactly, wanted, hobby, lobby, quit, carrying...","[0.02428521, 0.05687098, -0.0034518773, 0.0550...","[-0.01600385, 0.16626655, 0.031086037, 0.07007..."
249998,pen refill work expected,5,1,"[pen, refill, work, expected]","[-0.059318542, 0.15197754, -0.10107422, 0.0948...","[-0.13120882, 0.0047933124, -0.079583794, 0.03..."


## Binary Classification using FNN - average Word2Vec vectors

The model is a 3-layer feedforward network with two hidden layers, using ReLU activation and dropout.

In [95]:
model = nn.Sequential(
   nn.Linear(300, 128),
   nn.ReLU(),
   nn.Linear(128, 64),
   nn.ReLU(),
   nn.Linear(64, 2)
)

## Hyperparameters

In [96]:
learning_rate = 0.001

In [97]:
criterion = nn.CrossEntropyLoss()

In [98]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [99]:
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode='max',
    factor=0.1,
    patience=3,
    verbose=True
)

/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


## This function orchestrates the entire training process for a PyTorch model, handling data loading, forward and backward passes, optimization, and model saving.

In [100]:
#Binary
def train_model(model, train_data, train_labels, num_epochs=10, batch_size=32):
   train_data = torch.FloatTensor(train_data)
   train_labels = torch.LongTensor(train_labels.values.astype(int))

   train_dataset = list(zip(train_data, train_labels))
   train_loader = torch.utils.data.DataLoader(
       train_dataset, batch_size=batch_size, shuffle=True
   )

   best_acc = 0
   for epoch in range(num_epochs):
       model.train()
       total_loss = 0
       total_acc = 0

       for batch_data, batch_labels in train_loader:
           optimizer.zero_grad()
           outputs = model(batch_data)
           loss = criterion(outputs, batch_labels)
           loss.backward()
           torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
           optimizer.step()

           total_loss += loss.item()
           acc = accuracy(outputs, batch_labels)
           total_acc += acc

       avg_loss = total_loss/len(train_loader)
       avg_acc = total_acc/len(train_loader)
       print(f'Epoch {epoch+1}/{num_epochs}')
       print(f'Training Loss: {avg_loss:.4f}')
       print(f'Training Accuracy: {avg_acc:.4f}')

       scheduler.step(avg_acc)

       if avg_acc > best_acc:
           best_acc = avg_acc
           torch.save(model.state_dict(), 'best_model.pth')

## It determines how often the model's predictions match the actual labels.

In [101]:
def accuracy(outputs, labels):
    _, predicted = torch.max(outputs.data, 1)
    total = labels.size(0)
    correct = (predicted == labels).sum().item()
    return correct / total

## Training and Testing FNN For custom model (Binary)

In [102]:
train_model(model, X_train_custom, y_train_custom)

Epoch 1/10
Training Loss: 0.4698
Training Accuracy: 0.7973
Epoch 2/10
Training Loss: 0.4581
Training Accuracy: 0.8015
Epoch 3/10
Training Loss: 0.4554
Training Accuracy: 0.8017
Epoch 4/10
Training Loss: 0.4543
Training Accuracy: 0.8021
Epoch 5/10
Training Loss: 0.4534
Training Accuracy: 0.8020
Epoch 6/10
Training Loss: 0.4526
Training Accuracy: 0.8021
Epoch 7/10
Training Loss: 0.4526
Training Accuracy: 0.8023
Epoch 8/10
Training Loss: 0.4516
Training Accuracy: 0.8021
Epoch 9/10
Training Loss: 0.4513
Training Accuracy: 0.8019
Epoch 10/10
Training Loss: 0.4503
Training Accuracy: 0.8020


In [103]:
model.eval()
with torch.no_grad():
    X_test_tensor = torch.FloatTensor(X_test_custom)
    y_test_tensor = torch.LongTensor(y_test_custom.values)

    test_outputs = model(X_test_tensor)
    test_loss = criterion(test_outputs, y_test_tensor)
    test_acc_1 = accuracy(test_outputs, y_test_tensor)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_acc_1:.4f}')

Test Loss: 0.4535
Test Accuracy: 0.7995


## Training and Testing FNN For Pre-trained model (Binary)

In [104]:
train_model(model, X_train, y_train)

Epoch 1/10
Training Loss: 0.4696
Training Accuracy: 0.8013
Epoch 2/10
Training Loss: 0.4619
Training Accuracy: 0.8017
Epoch 3/10
Training Loss: 0.4599
Training Accuracy: 0.8021
Epoch 4/10
Training Loss: 0.4595
Training Accuracy: 0.8018
Epoch 5/10
Training Loss: 0.4585
Training Accuracy: 0.8019
Epoch 6/10
Training Loss: 0.4577
Training Accuracy: 0.8021
Epoch 7/10
Training Loss: 0.4572
Training Accuracy: 0.8020
Epoch 8/10
Training Loss: 0.4576
Training Accuracy: 0.8021
Epoch 9/10
Training Loss: 0.4575
Training Accuracy: 0.8019
Epoch 10/10
Training Loss: 0.4575
Training Accuracy: 0.8020


In [105]:
model.eval()
with torch.no_grad():
    X_test_tensor = torch.FloatTensor(X_test)
    y_test_tensor = torch.LongTensor(y_test.values)

    test_outputs = model(X_test_tensor)
    test_loss = criterion(test_outputs, y_test_tensor)
    test_acc_2 = accuracy(test_outputs, y_test_tensor)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_acc_2:.4f}')

Test Loss: 0.4634
Test Accuracy: 0.7982


## Train and Test split for Ternary Classification for custom model

In [106]:
X_train_custom_t, X_test_custom_t, y_train_custom_t, y_test_custom_t = train_test_split(np.stack(balanced_data_1['custom_embeddings'].values), balanced_data_1['sentiment'], test_size=0.2,random_state=40)

In [107]:
print(f"Training set shape: {X_train_custom_t.shape}, {y_train_custom_t.shape}")
print(f"Testing set shape: {X_test_custom_t.shape}, {y_test_custom_t.shape}")

Training set shape: (200000, 300), (200000,)
Testing set shape: (50000, 300), (50000,)


## Train and Test split for Ternary Classification for pretrained model

In [108]:
X_train_t, X_test_t, y_train_t, y_test_t = train_test_split(np.stack(balanced_data_1['Pre-trained embeddings'].values), balanced_data_1['sentiment'], test_size=0.2,random_state=40)

In [109]:
print(f"Training set shape: {X_train_t.shape}, {y_train_t.shape}")
print(f"Testing set shape: {X_test_t.shape}, {y_test_t.shape}")

Training set shape: (200000, 300), (200000,)
Testing set shape: (50000, 300), (50000,)


## Ternary Classification using FNN - average Word2Vec vectors

In [149]:
model_t = nn.Sequential(
   nn.Linear(300, 128),
   nn.ReLU(),
   nn.Linear(128, 64),
   nn.ReLU(),
   nn.Linear(64, 3)
)

In [152]:
lr = 0.001

In [153]:
optimizer = optim.Adam(model_t.parameters(), lr=lr, weight_decay=1e-5)

In [154]:
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2)

## The model is a 3-layer feedforward network with two hidden layers, using ReLU activation and dropout for ternary labels.

In [155]:
# Ternary
def train_model_t(model, train_data, train_labels, num_epochs=20, batch_size=64):
   train_data = torch.FloatTensor(train_data)
   train_labels = torch.LongTensor(train_labels.values.astype(int)) - 1

   train_dataset = list(zip(train_data, train_labels))
   train_loader = torch.utils.data.DataLoader(
       train_dataset, batch_size=batch_size, shuffle=True
   )

   best_acc = 0
   for epoch in range(num_epochs):
       model.train()
       total_loss = 0
       total_acc = 0

       for batch_data, batch_labels in train_loader:
           optimizer.zero_grad()
           outputs = model(batch_data)
           loss = criterion(outputs, batch_labels)
           loss.backward()
           torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
           optimizer.step()

           total_loss += loss.item()
           acc = accuracy(outputs, batch_labels)
           total_acc += acc

       avg_loss = total_loss/len(train_loader)
       avg_acc = total_acc/len(train_loader)
       print(f'Epoch {epoch+1}/{num_epochs}')
       print(f'Training Loss: {avg_loss:.4f}')
       print(f'Training Accuracy: {avg_acc:.4f}')

       scheduler.step(avg_acc)

       if avg_acc > best_acc:
           best_acc = avg_acc
           torch.save(model.state_dict(), 'best_model.pth')

## Training and testing FNN for custom model (ternary)

In [156]:
train_model_t(model_t, X_train_custom_t, y_train_custom_t)

Epoch 1/20
Training Loss: 0.7644
Training Accuracy: 0.6710
Epoch 2/20
Training Loss: 0.7272
Training Accuracy: 0.6876
Epoch 3/20
Training Loss: 0.7137
Training Accuracy: 0.6942
Epoch 4/20
Training Loss: 0.7047
Training Accuracy: 0.6983
Epoch 5/20
Training Loss: 0.6974
Training Accuracy: 0.7010
Epoch 6/20
Training Loss: 0.6912
Training Accuracy: 0.7036
Epoch 7/20
Training Loss: 0.6857
Training Accuracy: 0.7059
Epoch 8/20
Training Loss: 0.6806
Training Accuracy: 0.7089
Epoch 9/20
Training Loss: 0.6762
Training Accuracy: 0.7104
Epoch 10/20
Training Loss: 0.6720
Training Accuracy: 0.7124
Epoch 11/20
Training Loss: 0.6672
Training Accuracy: 0.7149
Epoch 12/20
Training Loss: 0.6642
Training Accuracy: 0.7152
Epoch 13/20
Training Loss: 0.6597
Training Accuracy: 0.7177
Epoch 14/20
Training Loss: 0.6567
Training Accuracy: 0.7188
Epoch 15/20
Training Loss: 0.6530
Training Accuracy: 0.7200
Epoch 16/20
Training Loss: 0.6493
Training Accuracy: 0.7219
Epoch 17/20
Training Loss: 0.6461
Training Accura

In [157]:
model_t.load_state_dict(torch.load('best_model.pth'))
model_t.eval()
with torch.no_grad():
   X_test_tensor = torch.FloatTensor(X_test_custom_t)
   y_test_tensor = torch.LongTensor(y_test_custom_t.values.astype(int)) - 1
   test_outputs = model_t(X_test_tensor)
   test_loss = criterion(test_outputs, y_test_tensor)
   test_acc_3 = accuracy(test_outputs, y_test_tensor)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_acc_3:.4f}')

<ipython-input-157-6457a6c41754>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_t.load_state_dict(torch.load('best_model.pth'))


Test Loss: 0.7118
Test Accuracy: 0.6978


## Training and testing FNN for pre-trained model (ternary)

In [158]:
train_model_t(model_t, X_train_t, y_train_t)

Epoch 1/20
Training Loss: 0.7972
Training Accuracy: 0.6554
Epoch 2/20
Training Loss: 0.7588
Training Accuracy: 0.6724
Epoch 3/20
Training Loss: 0.7437
Training Accuracy: 0.6789
Epoch 4/20
Training Loss: 0.7259
Training Accuracy: 0.6867
Epoch 5/20
Training Loss: 0.7186
Training Accuracy: 0.6901
Epoch 6/20
Training Loss: 0.7123
Training Accuracy: 0.6931
Epoch 7/20
Training Loss: 0.7020
Training Accuracy: 0.6972
Epoch 8/20
Training Loss: 0.6979
Training Accuracy: 0.6998
Epoch 9/20
Training Loss: 0.6949
Training Accuracy: 0.7011
Epoch 10/20
Training Loss: 0.6890
Training Accuracy: 0.7036
Epoch 11/20
Training Loss: 0.6869
Training Accuracy: 0.7045
Epoch 12/20
Training Loss: 0.6851
Training Accuracy: 0.7053
Epoch 13/20
Training Loss: 0.6818
Training Accuracy: 0.7071
Epoch 14/20
Training Loss: 0.6809
Training Accuracy: 0.7072
Epoch 15/20
Training Loss: 0.6798
Training Accuracy: 0.7079
Epoch 16/20
Training Loss: 0.6781
Training Accuracy: 0.7087
Epoch 17/20
Training Loss: 0.6776
Training Accura

In [159]:
model_t.load_state_dict(torch.load('best_model.pth'))
model_t.eval()
with torch.no_grad():
   X_test_tensor = torch.FloatTensor(X_test_t)
   y_test_tensor = torch.LongTensor(y_test_t.values.astype(int)) - 1
   test_outputs = model_t(X_test_tensor)
   test_loss = criterion(test_outputs, y_test_tensor)
   test_acc_4 = accuracy(test_outputs, y_test_tensor)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_acc_4:.4f}')

Test Loss: 0.7385
Test Accuracy: 0.6826


<ipython-input-159-ffd4f7496ea7>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_t.load_state_dict(torch.load('best_model.pth'))


In [116]:
balanced_data_1

,review_body,star_rating,sentiment,tokens,Pre-trained embeddings,custom_embeddings
0,returned card found quality poor flimsy inside...,1,2,"[returned, card, found, quality, poor, flimsy,...","[0.06468335, 0.06768478, -0.02350148, 0.075450...","[-0.00865445, 0.08607587, 0.010163504, 0.08254..."
1,waste money st laster shipped didnt work took ...,1,2,"[waste, money, st, laster, shipped, didnt, wor...","[0.041154534, 0.06830432, 0.029785682, 0.06836...","[-0.046187904, 0.12178713, 0.0056259334, 0.107..."
2,purchased allegedly smokeless ashtray house gu...,1,2,"[purchased, allegedly, smokeless, ashtray, hou...","[0.06975949, 0.04399525, 0.015591708, 0.076345...","[0.008813707, 0.17818084, -0.0925688, 0.121685..."
3,bought bunch supposedly compatible cartridge a...,1,2,"[bought, bunch, supposedly, compatible, cartri...","[-0.0014282226, 0.048101425, 0.03824692, 0.088...","[0.025674889, 0.034872424, -0.117311954, 0.030..."
4,stopped working even ten minute using,1,2,"[stopped, working, even, ten, minute, using]","[0.020268759, 0.03565216, 0.04871623, 0.083923...","[0.022188505, 0.13320307, -0.059528697, -0.051..."
...,...,...,...,...,...,...
249995,exactly advertised thank product timely delivery,5,1,"[exactly, advertised, thank, product, timely, ...","[-0.06542969, -0.02557373, -0.014322917, 0.007...","[-0.020638617, 0.12789576, -0.07443294, 0.0957..."
249996,looking projector hanger smallest drop since u...,5,1,"[looking, projector, hanger, smallest, drop, s...","[0.020986496, -0.013807727, -0.029896397, 0.08...","[0.0045630815, 0.110500075, 0.020787884, 0.121..."
249997,exactly wanted hobby lobby quit carrying glad ...,5,1,"[exactly, wanted, hobby, lobby, quit, carrying...","[0.02428521, 0.05687098, -0.0034518773, 0.0550...","[-0.01600385, 0.16626655, 0.031086037, 0.07007..."
249998,pen refill work expected,5,1,"[pen, refill, work, expected]","[-0.059318542, 0.15197754, -0.10107422, 0.0948...","[-0.13120882, 0.0047933124, -0.079583794, 0.03..."


In [117]:
model = Word2Vec.load("my_word2vec.model")

## A custom word embedding by taking up to 10 word vectors from a given model for a list of words (tokens).

In [118]:
def get_custom_embedding_10(tokens, model, vector_size=300, max_words=10):
    word_vectors = [model.wv[word] for word in tokens if word in model.wv]
    if len(word_vectors) > max_words:
        word_vectors = word_vectors[:max_words]
    else:
        padding_needed = max_words - len(word_vectors)
        word_vectors.extend([np.zeros(vector_size) for _ in range(padding_needed)])

    return np.concatenate(word_vectors)

In [119]:
balanced_data_1['custom_10_embeddings'] = balanced_data_1['tokens'].apply(lambda x: get_custom_embedding_10(x, model))

In [120]:
balanced_data_1

,review_body,star_rating,sentiment,tokens,Pre-trained embeddings,custom_embeddings,custom_10_embeddings
0,returned card found quality poor flimsy inside...,1,2,"[returned, card, found, quality, poor, flimsy,...","[0.06468335, 0.06768478, -0.02350148, 0.075450...","[-0.00865445, 0.08607587, 0.010163504, 0.08254...","[-0.091233484, 0.079264805, -0.052404094, 0.09..."
1,waste money st laster shipped didnt work took ...,1,2,"[waste, money, st, laster, shipped, didnt, wor...","[0.041154534, 0.06830432, 0.029785682, 0.06836...","[-0.046187904, 0.12178713, 0.0056259334, 0.107...","[0.04588635, -0.04905111, 0.14180414, 0.152214..."
2,purchased allegedly smokeless ashtray house gu...,1,2,"[purchased, allegedly, smokeless, ashtray, hou...","[0.06975949, 0.04399525, 0.015591708, 0.076345...","[0.008813707, 0.17818084, -0.0925688, 0.121685...","[-0.12329905, 0.18327253, -0.20252548, 0.15969..."
3,bought bunch supposedly compatible cartridge a...,1,2,"[bought, bunch, supposedly, compatible, cartri...","[-0.0014282226, 0.048101425, 0.03824692, 0.088...","[0.025674889, 0.034872424, -0.117311954, 0.030...","[-0.13236512, 0.28369874, -0.20264532, 0.08766..."
4,stopped working even ten minute using,1,2,"[stopped, working, even, ten, minute, using]","[0.020268759, 0.03565216, 0.04871623, 0.083923...","[0.022188505, 0.13320307, -0.059528697, -0.051...","[-0.07354360073804855, 0.175058975815773, 0.09..."
...,...,...,...,...,...,...,...
249995,exactly advertised thank product timely delivery,5,1,"[exactly, advertised, thank, product, timely, ...","[-0.06542969, -0.02557373, -0.014322917, 0.007...","[-0.020638617, 0.12789576, -0.07443294, 0.0957...","[0.1383015215396881, 0.17258672416210175, -0.1..."
249996,looking projector hanger smallest drop since u...,5,1,"[looking, projector, hanger, smallest, drop, s...","[0.020986496, -0.013807727, -0.029896397, 0.08...","[0.0045630815, 0.110500075, 0.020787884, 0.121...","[-0.14100368, 0.13715553, -0.006589446, 0.0113..."
249997,exactly wanted hobby lobby quit carrying glad ...,5,1,"[exactly, wanted, hobby, lobby, quit, carrying...","[0.02428521, 0.05687098, -0.0034518773, 0.0550...","[-0.01600385, 0.16626655, 0.031086037, 0.07007...","[0.13830152, 0.17258672, -0.17428407, 0.168998..."
249998,pen refill work expected,5,1,"[pen, refill, work, expected]","[-0.059318542, 0.15197754, -0.10107422, 0.0948...","[-0.13120882, 0.0047933124, -0.079583794, 0.03...","[-0.3010120987892151, -0.09458253532648087, 0...."


## This code creates fixed-size (10) sentence embeddings by combining word vectors or padding with zeros, storing the result in a new DataFrame column.

In [121]:
def get_sentence_embedding_10(tokens, max_words=10):
    word_vectors = [wv[word] for word in tokens if word in wv]
    if len(word_vectors) > max_words:
        word_vectors = word_vectors[:max_words]
    else:
        padding_needed = max_words - len(word_vectors)
        word_vectors.extend([np.zeros(300) for _ in range(padding_needed)])
    return np.concatenate(word_vectors)

balanced_data_1['Pre-trained 10 embeddings'] = balanced_data_1['tokens'].apply(get_sentence_embedding_10)

In [122]:
balanced_data_1

,review_body,star_rating,sentiment,tokens,Pre-trained embeddings,custom_embeddings,custom_10_embeddings,Pre-trained 10 embeddings
0,returned card found quality poor flimsy inside...,1,2,"[returned, card, found, quality, poor, flimsy,...","[0.06468335, 0.06768478, -0.02350148, 0.075450...","[-0.00865445, 0.08607587, 0.010163504, 0.08254...","[-0.091233484, 0.079264805, -0.052404094, 0.09...","[0.12402344, 0.18945312, -0.14648438, -0.01635..."
1,waste money st laster shipped didnt work took ...,1,2,"[waste, money, st, laster, shipped, didnt, wor...","[0.041154534, 0.06830432, 0.029785682, 0.06836...","[-0.046187904, 0.12178713, 0.0056259334, 0.107...","[0.04588635, -0.04905111, 0.14180414, 0.152214...","[-0.19824219, 0.38867188, 0.40234375, 0.181640..."
2,purchased allegedly smokeless ashtray house gu...,1,2,"[purchased, allegedly, smokeless, ashtray, hou...","[0.06975949, 0.04399525, 0.015591708, 0.076345...","[0.008813707, 0.17818084, -0.0925688, 0.121685...","[-0.12329905, 0.18327253, -0.20252548, 0.15969...","[0.068847656, -0.13769531, -0.13671875, 0.1738..."
3,bought bunch supposedly compatible cartridge a...,1,2,"[bought, bunch, supposedly, compatible, cartri...","[-0.0014282226, 0.048101425, 0.03824692, 0.088...","[0.025674889, 0.034872424, -0.117311954, 0.030...","[-0.13236512, 0.28369874, -0.20264532, 0.08766...","[0.16699219, -0.05419922, -0.087402344, 0.0196..."
4,stopped working even ten minute using,1,2,"[stopped, working, even, ten, minute, using]","[0.020268759, 0.03565216, 0.04871623, 0.083923...","[0.022188505, 0.13320307, -0.059528697, -0.051...","[-0.07354360073804855, 0.175058975815773, 0.09...","[0.010986328125, 0.05078125, 0.22265625, 0.024..."
...,...,...,...,...,...,...,...,...
249995,exactly advertised thank product timely delivery,5,1,"[exactly, advertised, thank, product, timely, ...","[-0.06542969, -0.02557373, -0.014322917, 0.007...","[-0.020638617, 0.12789576, -0.07443294, 0.0957...","[0.1383015215396881, 0.17258672416210175, -0.1...","[0.2158203125, -0.1357421875, 0.2119140625, -0..."
249996,looking projector hanger smallest drop since u...,5,1,"[looking, projector, hanger, smallest, drop, s...","[0.020986496, -0.013807727, -0.029896397, 0.08...","[0.0045630815, 0.110500075, 0.020787884, 0.121...","[-0.14100368, 0.13715553, -0.006589446, 0.0113...","[0.027832031, 0.25585938, 0.15820312, -0.04809..."
249997,exactly wanted hobby lobby quit carrying glad ...,5,1,"[exactly, wanted, hobby, lobby, quit, carrying...","[0.02428521, 0.05687098, -0.0034518773, 0.0550...","[-0.01600385, 0.16626655, 0.031086037, 0.07007...","[0.13830152, 0.17258672, -0.17428407, 0.168998...","[0.21582031, -0.13574219, 0.21191406, -0.01202..."
249998,pen refill work expected,5,1,"[pen, refill, work, expected]","[-0.059318542, 0.15197754, -0.10107422, 0.0948...","[-0.13120882, 0.0047933124, -0.079583794, 0.03...","[-0.3010120987892151, -0.09458253532648087, 0....","[0.001495361328125, 0.0615234375, -0.224609375..."


In [123]:
X_train_custom_10, X_test_custom_10, y_train_custom_10, y_test_custom_10 = train_test_split(np.stack(balanced_data_1['custom_10_embeddings'].values), balanced_data_1['sentiment'].isin([1, 2]), test_size=0.2,random_state=40)

In [124]:
X_train_10, X_test_10, y_train_10, y_test_10 = train_test_split(np.stack(balanced_data_1['Pre-trained 10 embeddings'].values), balanced_data_1['sentiment'].isin([1, 2]), test_size=0.2,random_state=40)

## Binary Classification using FNN - 10 Word2Vec vectors

In [164]:
model_10 = nn.Sequential(
   nn.Linear(3000, 128),
   nn.ReLU(),
   nn.Linear(128, 64),
   nn.ReLU(),
   nn.Linear(64, 2)
)

In [165]:
lr = 0.001
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_10.parameters(), lr=lr, weight_decay=1e-5)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2)

In [168]:
#Binary
def train_model(model, train_data, train_labels, num_epochs=20, batch_size=64):
   train_data = torch.FloatTensor(train_data)
   train_labels = torch.LongTensor(train_labels.values.astype(int))

   train_dataset = list(zip(train_data, train_labels))
   train_loader = torch.utils.data.DataLoader(
       train_dataset, batch_size=batch_size, shuffle=True
   )

   best_acc = 0
   for epoch in range(num_epochs):
       model.train()
       total_loss = 0
       total_acc = 0

       for batch_data, batch_labels in train_loader:
           optimizer.zero_grad()
           outputs = model(batch_data)
           loss = criterion(outputs, batch_labels)
           loss.backward()
           torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
           optimizer.step()

           total_loss += loss.item()
           acc = accuracy(outputs, batch_labels)
           total_acc += acc

       avg_loss = total_loss/len(train_loader)
       avg_acc = total_acc/len(train_loader)
       print(f'Epoch {epoch+1}/{num_epochs}')
       print(f'Training Loss: {avg_loss:.4f}')
       print(f'Training Accuracy: {avg_acc:.4f}')

       scheduler.step(avg_acc)

       if avg_acc > best_acc:
           best_acc = avg_acc
           torch.save(model.state_dict(), 'best_model.pth')

## Training and Testing FNN for Custom Model (Binary)

In [169]:
train_model(model_10, X_train_custom_10, y_train_custom_10)

Epoch 1/20
Training Loss: 0.4734
Training Accuracy: 0.8015
Epoch 2/20
Training Loss: 0.4592
Training Accuracy: 0.8035
Epoch 3/20
Training Loss: 0.4479
Training Accuracy: 0.8058
Epoch 4/20
Training Loss: 0.4316
Training Accuracy: 0.8100
Epoch 5/20
Training Loss: 0.4075
Training Accuracy: 0.8175
Epoch 6/20
Training Loss: 0.3738
Training Accuracy: 0.8306
Epoch 7/20
Training Loss: 0.3372
Training Accuracy: 0.8484
Epoch 8/20
Training Loss: 0.2988
Training Accuracy: 0.8685
Epoch 9/20
Training Loss: 0.2642
Training Accuracy: 0.8855
Epoch 10/20
Training Loss: 0.2341
Training Accuracy: 0.9002
Epoch 11/20
Training Loss: 0.2103
Training Accuracy: 0.9127
Epoch 12/20
Training Loss: 0.1886
Training Accuracy: 0.9224
Epoch 13/20
Training Loss: 0.1707
Training Accuracy: 0.9312
Epoch 14/20
Training Loss: 0.1566
Training Accuracy: 0.9363
Epoch 15/20
Training Loss: 0.1467
Training Accuracy: 0.9412
Epoch 16/20
Training Loss: 0.1357
Training Accuracy: 0.9457
Epoch 17/20
Training Loss: 0.1269
Training Accura

In [170]:
model_10.eval()
with torch.no_grad():
    X_test_tensor = torch.FloatTensor(X_test_custom_10)
    y_test_tensor = torch.LongTensor(y_test_custom_10.values)

    test_outputs = model_10(X_test_tensor)
    test_loss = criterion(test_outputs, y_test_tensor)
    test_acc_5 = accuracy(test_outputs, y_test_tensor)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_acc_5:.4f}')

Test Loss: 1.5602
Test Accuracy: 0.7198


## Training and Testing FNN for Pre-trained Model (Binary)

In [171]:
train_model(model_10, X_train_10, y_train_10)

Epoch 1/20
Training Loss: 0.5104
Training Accuracy: 0.7996
Epoch 2/20
Training Loss: 0.4666
Training Accuracy: 0.8024
Epoch 3/20
Training Loss: 0.4576
Training Accuracy: 0.8043
Epoch 4/20
Training Loss: 0.4297
Training Accuracy: 0.8112
Epoch 5/20
Training Loss: 0.3889
Training Accuracy: 0.8270
Epoch 6/20
Training Loss: 0.3386
Training Accuracy: 0.8518
Epoch 7/20
Training Loss: 0.2611
Training Accuracy: 0.8913
Epoch 8/20
Training Loss: 0.2184
Training Accuracy: 0.9120
Epoch 9/20
Training Loss: 0.1810
Training Accuracy: 0.9297
Epoch 10/20
Training Loss: 0.1324
Training Accuracy: 0.9531
Epoch 11/20
Training Loss: 0.1097
Training Accuracy: 0.9634
Epoch 12/20
Training Loss: 0.0903
Training Accuracy: 0.9704
Epoch 13/20
Training Loss: 0.0732
Training Accuracy: 0.9769
Epoch 14/20
Training Loss: 0.0589
Training Accuracy: 0.9818
Epoch 15/20
Training Loss: 0.0478
Training Accuracy: 0.9856
Epoch 16/20
Training Loss: 0.0401
Training Accuracy: 0.9876
Epoch 17/20
Training Loss: 0.0348
Training Accura

In [172]:
model_10.eval()
with torch.no_grad():
    X_test_tensor = torch.FloatTensor(X_test_10)
    y_test_tensor = torch.LongTensor(y_test_10.values)

    test_outputs = model_10(X_test_tensor)
    test_loss = criterion(test_outputs, y_test_tensor)
    test_acc_6 = accuracy(test_outputs, y_test_tensor)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_acc_6:.4f}')

Test Loss: 3.1063
Test Accuracy: 0.7242


## Ternary Classification using FNN - 10 Word2Vec vectors

In [173]:
model_10_t = nn.Sequential(
   nn.Linear(3000, 128),
   nn.ReLU(),
   nn.Linear(128, 64),
   nn.ReLU(),
   nn.Linear(64, 3)
)

In [174]:
lr = 0.001
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_10_t.parameters(), lr=lr, weight_decay=1e-5)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2)

In [175]:
# Ternary
def train_model_t(model, train_data, train_labels, num_epochs=20, batch_size=64):
   train_data = torch.FloatTensor(train_data)
   train_labels = torch.LongTensor(train_labels.values.astype(int)) - 1

   train_dataset = list(zip(train_data, train_labels))
   train_loader = torch.utils.data.DataLoader(
       train_dataset, batch_size=batch_size, shuffle=True
   )

   best_acc = 0
   for epoch in range(num_epochs):
       model.train()
       total_loss = 0
       total_acc = 0

       for batch_data, batch_labels in train_loader:
           optimizer.zero_grad()
           outputs = model(batch_data)
           loss = criterion(outputs, batch_labels)
           loss.backward()
           torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
           optimizer.step()

           total_loss += loss.item()
           acc = accuracy(outputs, batch_labels)
           total_acc += acc

       avg_loss = total_loss/len(train_loader)
       avg_acc = total_acc/len(train_loader)
       print(f'Epoch {epoch+1}/{num_epochs}')
       print(f'Training Loss: {avg_loss:.4f}')
       print(f'Training Accuracy: {avg_acc:.4f}')

       scheduler.step(avg_acc)

       if avg_acc > best_acc:
           best_acc = avg_acc
           torch.save(model.state_dict(), 'best_model.pth')

## Training and Testing FNN for custom model (Ternary)

In [176]:
X_train_custom_10_t, X_test_custom_10_t, y_train_custom_10_t, y_test_custom_10_t = train_test_split(np.stack(balanced_data_1['custom_10_embeddings'].values), balanced_data_1['sentiment'], test_size=0.2,random_state=40)

In [177]:
train_model_t(model_10_t, X_train_custom_10_t, y_train_custom_10_t)

Epoch 1/20
Training Loss: 0.8353
Training Accuracy: 0.6290
Epoch 2/20
Training Loss: 0.7852
Training Accuracy: 0.6562
Epoch 3/20
Training Loss: 0.7446
Training Accuracy: 0.6784
Epoch 4/20
Training Loss: 0.6915
Training Accuracy: 0.7050
Epoch 5/20
Training Loss: 0.6282
Training Accuracy: 0.7353
Epoch 6/20
Training Loss: 0.5596
Training Accuracy: 0.7666
Epoch 7/20
Training Loss: 0.4964
Training Accuracy: 0.7948
Epoch 8/20
Training Loss: 0.4411
Training Accuracy: 0.8192
Epoch 9/20
Training Loss: 0.3960
Training Accuracy: 0.8391
Epoch 10/20
Training Loss: 0.3595
Training Accuracy: 0.8545
Epoch 11/20
Training Loss: 0.3285
Training Accuracy: 0.8688
Epoch 12/20
Training Loss: 0.3010
Training Accuracy: 0.8803
Epoch 13/20
Training Loss: 0.2814
Training Accuracy: 0.8891
Epoch 14/20
Training Loss: 0.2622
Training Accuracy: 0.8978
Epoch 15/20
Training Loss: 0.2458
Training Accuracy: 0.9042
Epoch 16/20
Training Loss: 0.2305
Training Accuracy: 0.9105
Epoch 17/20
Training Loss: 0.2183
Training Accura

In [178]:
model_10_t.eval()
with torch.no_grad():
    X_test_tensor = torch.FloatTensor(X_test_custom_10_t)
    y_test_tensor = torch.LongTensor(y_test_custom_10_t.values) - 1

    test_outputs = model_10_t(X_test_tensor)
    test_loss = criterion(test_outputs, y_test_tensor)
    test_acc_7 = accuracy(test_outputs, y_test_tensor)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_acc_7:.4f}')

Test Loss: 2.6515
Test Accuracy: 0.5713


## Training and Testing FNN for pre-trained model (Ternary)

In [179]:
X_train_10_t, X_test_10_t, y_train_10_t, y_test_10_t = train_test_split(np.stack(balanced_data_1['Pre-trained 10 embeddings'].values), balanced_data_1['sentiment'], test_size=0.2,random_state=40)

In [180]:
train_model_t(model_10_t, X_train_10_t, y_train_10_t)

Epoch 1/20
Training Loss: 0.8982
Training Accuracy: 0.6046
Epoch 2/20
Training Loss: 0.8362
Training Accuracy: 0.6296
Epoch 3/20
Training Loss: 0.8069
Training Accuracy: 0.6451
Epoch 4/20
Training Loss: 0.7375
Training Accuracy: 0.6826
Epoch 5/20
Training Loss: 0.6740
Training Accuracy: 0.7152
Epoch 6/20
Training Loss: 0.6103
Training Accuracy: 0.7479
Epoch 7/20
Training Loss: 0.5193
Training Accuracy: 0.7935
Epoch 8/20
Training Loss: 0.4736
Training Accuracy: 0.8142
Epoch 9/20
Training Loss: 0.4335
Training Accuracy: 0.8337
Epoch 10/20
Training Loss: 0.3753
Training Accuracy: 0.8613
Epoch 11/20
Training Loss: 0.3492
Training Accuracy: 0.8727
Epoch 12/20
Training Loss: 0.3259
Training Accuracy: 0.8830
Epoch 13/20
Training Loss: 0.2935
Training Accuracy: 0.8977
Epoch 14/20
Training Loss: 0.2803
Training Accuracy: 0.9031
Epoch 15/20
Training Loss: 0.2683
Training Accuracy: 0.9080
Epoch 16/20
Training Loss: 0.2518
Training Accuracy: 0.9152
Epoch 17/20
Training Loss: 0.2453
Training Accura

In [181]:
model_10_t.eval()
with torch.no_grad():
    X_test_tensor = torch.FloatTensor(X_test_10_t)
    y_test_tensor = torch.LongTensor(y_test_10_t.values.astype(int)) - 1

    test_outputs = model_10_t(X_test_tensor)
    test_loss = criterion(test_outputs, y_test_tensor)
    test_acc_8 = accuracy(test_outputs, y_test_tensor)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_acc_8:.4f}')

Test Loss: 1.8413
Test Accuracy: 0.5683


# ***Convolutional Neural Networks***

## A SentimentCNN model for text classification. It uses convolutional layers to extract features from input word embeddings, applies pooling to reduce dimensions, and finally uses a fully connected layer for classification into sentiment classes (e.g., positive or negative).

In [138]:
class SentimentCNN(nn.Module):
    def __init__(self, vector_size=300, max_length=50, num_classes=2):
        super(SentimentCNN, self).__init__()

        self.conv1 = nn.Conv1d(vector_size, 50, kernel_size=3, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool1d(2)

        self.conv2 = nn.Conv1d(50, 10, kernel_size=3, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool1d(2)

        self.flat_size = 10 * (max_length // 4)
        self.fc = nn.Linear(self.flat_size, num_classes)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = x.transpose(1, 2)
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        x = x.view(x.size(0), -1)
        x = self.dropout(x)
        x = self.fc(x)
        return x

## This code defines a ReviewDataset class for handling text data in sentiment analysis tasks. It converts input texts into fixed-length word embedding sequences using a Word2Vec model and pairs them with their labels. Texts longer than the max length are truncated, and shorter ones are padded with zeros.

In [139]:
class ReviewDataset(Dataset):
    def __init__(self, texts, labels, word2vec_model, max_length=50, vector_size=300, is_ternary=False):
        self.texts = texts.values if hasattr(texts, 'values') else texts

        if is_ternary:
            labels = labels - 1

        self.labels = torch.tensor(labels.values if hasattr(labels, 'values') else labels, dtype=torch.long)
        self.word2vec_model = word2vec_model.wv if hasattr(word2vec_model, 'wv') else word2vec_model
        self.max_length = max_length
        self.vector_size = vector_size

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        vectors = self._text_to_vectors(text)
        return vectors, self.labels[idx]

    def _text_to_vectors(self, text):
        vectors = []
        for word in text.split():
            try:
                if word in self.word2vec_model:
                    vectors.append(self.word2vec_model[word])
            except TypeError:
                continue

        if not vectors:
            return torch.zeros((self.max_length, self.vector_size))

        vectors = np.array(vectors)
        vectors = torch.tensor(vectors, dtype=torch.float32)

        if len(vectors) > self.max_length:
            vectors = vectors[:self.max_length]
        else:
            padding = torch.zeros((self.max_length - len(vectors), self.vector_size))
            vectors = torch.cat([vectors, padding])

        return vectors

## This function trains a PyTorch model using the Adam optimizer and cross-entropy loss, evaluates it on a test set after each epoch, and returns the best test accuracy.

In [140]:
def train_model(model, train_loader, test_loader, num_epochs=5, learning_rate=0.001):
    device = 'cpu'
    model = model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    best_accuracy = 0

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0

        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)

            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        model.eval()
        predictions = []
        actual = []

        with torch.no_grad():
            for data, target in test_loader:
                data, target = data.to(device), target.to(device)
                output = model(data)
                pred = output.argmax(dim=1)
                predictions.extend(pred.numpy())
                actual.extend(target.numpy())

        accuracy = accuracy_score(actual, predictions)
        print(f'Epoch {epoch+1}/{num_epochs}')
        print(f'Average Loss: {total_loss/len(train_loader):.4f}')
        print(f'Test Accuracy: {accuracy:.4f}\n')

        if accuracy > best_accuracy:
            best_accuracy = accuracy

    return best_accuracy

## This function evaluates binary and ternary sentiment classification using both pretrained and custom models. It creates datasets, loaders, and trains models for each setup, storing the results (e.g., accuracy) in a dictionary for comparison.

In [141]:
def evaluate_all_models(X_train_binary, y_train_binary, X_test_binary, y_test_binary,
                       X_train_ternary, y_train_ternary, X_test_ternary, y_test_ternary,
                       pretrained_model, custom_model, batch_size=64):

    results = {}

    print("\nTraining Binary Classification with Pretrained Model")
    binary_model_pretrained = SentimentCNN(num_classes=2)
    train_dataset = ReviewDataset(X_train_binary, y_train_binary, pretrained_model, is_ternary=False)
    test_dataset = ReviewDataset(X_test_binary, y_test_binary, pretrained_model, is_ternary=False)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)
    results['binary_pretrained'] = train_model(binary_model_pretrained, train_loader, test_loader)

    print("\nTraining Binary Classification with Custom Model")
    binary_model_custom = SentimentCNN(num_classes=2)
    train_dataset = ReviewDataset(X_train_binary, y_train_binary, custom_model, is_ternary=False)
    test_dataset = ReviewDataset(X_test_binary, y_test_binary, custom_model, is_ternary=False)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)
    results['binary_custom'] = train_model(binary_model_custom, train_loader, test_loader)

    print("\nTraining Ternary Classification with Pretrained Model")
    ternary_model_pretrained = SentimentCNN(num_classes=3)
    train_dataset = ReviewDataset(X_train_ternary, y_train_ternary, pretrained_model, is_ternary=True)
    test_dataset = ReviewDataset(X_test_ternary, y_test_ternary, pretrained_model, is_ternary=True)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)
    results['ternary_pretrained'] = train_model(ternary_model_pretrained, train_loader, test_loader)

    print("\nTraining Ternary Classification with Custom Model")
    ternary_model_custom = SentimentCNN(num_classes=3)
    train_dataset = ReviewDataset(X_train_ternary, y_train_ternary, custom_model, is_ternary=True)
    test_dataset = ReviewDataset(X_test_ternary, y_test_ternary, custom_model, is_ternary=True)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)
    results['ternary_custom'] = train_model(ternary_model_custom, train_loader, test_loader)

    return results

In [142]:
X_train_binary, X_test_binary, y_train_binary, y_test_binary = train_test_split(balanced_data_1['review_body'], balanced_data_1['sentiment'].isin([1, 2]), test_size=0.2,random_state=40)

In [143]:
X_train_ternary, X_test_ternary, y_train_ternary, y_test_ternary = train_test_split(balanced_data_1['review_body'], balanced_data_1['sentiment'], test_size=0.2,random_state=40)

In [144]:
accuracies = evaluate_all_models(
    X_train_binary, y_train_binary, X_test_binary, y_test_binary,
    X_train_ternary, y_train_ternary, X_test_ternary, y_test_ternary,
    wv,
    model
)

binary_pretrained_acc = accuracies['binary_pretrained']
binary_custom_acc = accuracies['binary_custom']
ternary_pretrained_acc = accuracies['ternary_pretrained']
ternary_custom_acc = accuracies['ternary_custom']

print(f"Binary Classification Using CNN - Pretrained Model: {binary_pretrained_acc:.4f}")
print(f"Binary Classification Using CNN - Custom Model: {binary_custom_acc:.4f}")
print(f"Ternary Classification Using CNN - Pretrained Model: {ternary_pretrained_acc:.4f}")
print(f"Ternary Classification Using CNN - Custom Model: {ternary_custom_acc:.4f}")


Training Binary Classification with Pretrained Model
Epoch 1/5
Average Loss: 0.4746
Test Accuracy: 0.7970

Epoch 2/5
Average Loss: 0.4589
Test Accuracy: 0.7970

Epoch 3/5
Average Loss: 0.4513
Test Accuracy: 0.7970

Epoch 4/5
Average Loss: 0.4440
Test Accuracy: 0.7970

Epoch 5/5
Average Loss: 0.4365
Test Accuracy: 0.7965


Training Binary Classification with Custom Model
Epoch 1/5
Average Loss: 0.4731
Test Accuracy: 0.7970

Epoch 2/5
Average Loss: 0.4585
Test Accuracy: 0.7970

Epoch 3/5
Average Loss: 0.4522
Test Accuracy: 0.7970

Epoch 4/5
Average Loss: 0.4467
Test Accuracy: 0.7970

Epoch 5/5
Average Loss: 0.4400
Test Accuracy: 0.7970


Training Ternary Classification with Pretrained Model
Epoch 1/5
Average Loss: 0.7986
Test Accuracy: 0.6896

Epoch 2/5
Average Loss: 0.7344
Test Accuracy: 0.6978

Epoch 3/5
Average Loss: 0.7137
Test Accuracy: 0.7019

Epoch 4/5
Average Loss: 0.6990
Test Accuracy: 0.7035

Epoch 5/5
Average Loss: 0.6891
Test Accuracy: 0.7051


Training Ternary Classificatio

# ***FINAL 16 ACCURACIES***

In [182]:
import pandas as pd

data = {
    'Model': [
        'Perceptron - Custom Model',
        'Perceptron - Pre-trained Model',
        'Perceptron - TFIDF',
        'SVM - Pre-trained Model',
        'SVM - Custom Model',
        'SVM - TFIDF',
        'FNN for avg word2vec (Custom) - Binary',
        'FNN for avg word2vec (Pre-trained) - Binary',
        'FNN for avg word2vec (Custom) - Ternary',
        'FNN for avg word2vec (Pre-trained) - Ternary',
        'FNN for avg word2vec (Custom) - Binary (10)',
        'FNN for avg word2vec (Pre-trained) - Binary (10)',
        'FNN for avg word2vec (Custom) - Ternary (10)',
        'FNN for avg word2vec (Pre-trained) - Ternary (10)',
        'CNN - Pre-trained Model - Binary',
        'CNN - Custom Model - Binary',
        'CNN - Pre-trained Model - Ternary',
        'CNN - Custom Model - Ternary'
    ],
    'Accuracy': [
        test_accuracy_Perceptron_Custom_Model,
        test_accuracy_Perceptron_Pretrained_Model,
        test_accuracy_Perceptron_TFIDF,
        test_accuracy_SVM_Pretrained_Model,
        test_accuracy_SVM_Custom_Model,
        test_accuracy_SVM_TFIDF,
        test_acc_1,
        test_acc_2,
        test_acc_3,
        test_acc_4,
        test_acc_5,
        test_acc_6,
        test_acc_7,
        test_acc_8,
        binary_pretrained_acc,
        binary_custom_acc,
        ternary_pretrained_acc,
        ternary_custom_acc
    ]
}

df_accuracies = pd.DataFrame(data)

df_accuracies['Accuracy'] = df_accuracies['Accuracy'].round(4)

print(df_accuracies)

                                                Model  Accuracy
0                           Perceptron - Custom Model    0.7868
1                      Perceptron - Pre-trained Model    0.7643
2                                  Perceptron - TFIDF    0.7151
3                             SVM - Pre-trained Model    0.7975
4                                  SVM - Custom Model    0.7971
5                                         SVM - TFIDF    0.7961
6              FNN for avg word2vec (Custom) - Binary    0.7995
7         FNN for avg word2vec (Pre-trained) - Binary    0.7982
8             FNN for avg word2vec (Custom) - Ternary    0.6978
9        FNN for avg word2vec (Pre-trained) - Ternary    0.6826
10        FNN for avg word2vec (Custom) - Binary (10)    0.7198
11   FNN for avg word2vec (Pre-trained) - Binary (10)    0.7242
12       FNN for avg word2vec (Custom) - Ternary (10)    0.5713
13  FNN for avg word2vec (Pre-trained) - Ternary (10)    0.5683
14                   CNN - Pre-trained M

# ***References***

* https://www.kaggle.com/mishra1993/pytorch-multi-layer-perceptron-mnist
* https://radimrehurek.com/gensim/auto_examples/tutorials/run_word2vec.html
* https://www.geeksforgeeks.org/multi-layer-perceptron-learning-in-tensorflow/
* https://medium.com/@RaghavPrabhu/understanding-of-convolutional-neural-network-cnn-deep-learning-99760835f148
* https://medium.com/@gauravtailor43/googles-trained-word2vec-model-in-python-ab9b2df09af2